In [ ]:
!git clone https://github.com/nd7141/icml2020.git

fatal: destination path 'icml2020' already exists and is not an empty directory.


In [ ]:
%cd icml2020/

/content/icml2020


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np

In [ ]:
!pip install qgrid

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from collections import defaultdict as ddict, Counter
from itertools import chain

import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import qgrid

In [ ]:
# (only for collab)
if False:
    !git clone https://github.com/nd7141/icml2020.git
    !cp icml2020/university2.csv .
    !cp icml2020/icml_2020_papers.txt .
    !ls

In [ ]:
filename='neurips_2020_accepted.txt'
papers = ddict(list)
authors = ddict(list)
affiliations = ddict(set)
author2affiliation = ddict(list)
paper2authors = ddict(set)
paper2affiliations = ddict(set)
from unis import unis
with open(filename, encoding="utf8") as f:
    i = 0
    while i < 500:
        paper_title = next(f)
        paper_authors = next(f)

        author_splitted = paper_authors.split('·')
        paper2authors[paper_title] = author_splitted

        for author in author_splitted:
#                 full_name = author[:author.rfind("(")].strip()
            full_name = author.strip()
            affiliation = author[author.rfind("(")+1:author.rfind(")")]
            if affiliation in unis:
                affiliation = unis[affiliation]
            for sep in [',', ' and', ' &', '/', '+', ';', ')']:
                affiliation = affiliation.split(sep)[0]
            
            print(full_name, affiliation)
        next(f)
        print()
        i += 1

Natalia Neverova (Facebook AI Research) Facebook AI Research
David Novotny (Facebook AI Research) Facebook AI Research
Marc Szafraniec (Facebook AI Research) Facebook AI Research
Vasil Khalidov (Facebook AI Research) Facebook AI Research
Patrick Labatut (Facebook AI Research) Facebook AI Research
Andrea Vedaldi (University of Oxford / Facebook AI Research) University of Oxford 

Ranganath Krishnan (Intel Labs) Intel
Omesh Tickoo (Intel) Intel

Yijun Li (Adobe Research) Adobe
Richard Zhang (Adobe) Adobe
Jingwan (Cynthia) Lu (Adobe Research) Adobe
Eli Shechtman (Adobe Research, US) Adobe Research

Umut Simsekli (Institut Polytechnique de Paris/ University of Oxford) Institut Polytechnique de Paris
Ozan Sener (Intel Labs) Intel
George Deligiannidis (Oxford) Oxford
Murat Erdogdu (University of Toronto) University of Toronto

Valentin De Bortoli (ENS Paris-Saclay) ENS Paris-Saclay
Alain Durmus (ENS Paris Saclay) ENS Paris Saclay
Xavier Fontaine (ENS Paris-Saclay) ENS Paris-Saclay
Umut Simse

In [ ]:
from unis import unis 

def read_papers(filename):
    papers = ddict(list)
    authors = ddict(list)
    affiliations = ddict(set)
    author2affiliation = ddict(list)
    paper2authors = ddict(set)
    paper2affiliations = ddict(set)
    with open(filename, encoding="utf8") as f:
        while True:
            paper_title = next(f)
            paper_authors = next(f)
            
            if len(paper_authors) < 10:
                print(paper_title)
                break

            author_splitted = paper_authors.split('·')
            paper2authors[paper_title] = author_splitted

            for author in author_splitted:
#                 full_name = author[:author.rfind("(")].strip()
                full_name = author.strip()
                affiliation = author[author.rfind("(")+1:author.rfind(")")]
                
                if affiliation in unis:
                    affiliation = unis[affiliation]
                for sep in [',', ' and', ' &', '/', '+', ';', ')']:
                    if not (sep == ',' and 'university of california' in affiliation.lower()):
                        affiliation = affiliation.split(sep)[0]
                        
            
                

                paper2affiliations[paper_title].add(affiliation)

                papers[paper_title.strip()].append(full_name)
                authors[full_name.strip()].append(paper_title)
                affiliations[affiliation.strip()].add(paper_title)
                author2affiliation[full_name.strip()].append(affiliation)
            try:
                next(f)
            except StopIteration:
                return papers, authors, affiliations, author2affiliation, paper2authors, paper2affiliations


def plot_bar(top_orgs, title='Number of papers', width=1200, height=800, xname='country', yname='papers',
                          xaxis_title="", yaxis_title="# papers", tickangle=45, tickmode='auto', margin_bottom=200,
                          save_html=False,
                          fontsize=20, title_y=0.98,
                          ):
    fig = px.bar(top_orgs, x=xname, y=yname, height=height, width=width, text='papers')
    fig.update_layout(
        title={
            'text': title,
            'y': title_y,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        font=dict(
            #         family="Courier New",
            size=fontsize,
            color="black"
        ),
        margin=dict(b=margin_bottom),
        xaxis=dict(tickangle=tickangle,
                   tickmode='linear'),

    )
    fig.update_traces(marker_color='blue')
    #     fig.update_traces(textposition='inside')

    if save_html:
        pio.write_html(fig, file='index.html', auto_open=True)
    fig.show()

papers, authors, affiliations, author2affiliation, paper2authors, paper2affiliations = read_papers(filename='neurips_2020_accepted.txt')

def read_mapping(filename='university2.csv'):
    uni = pd.read_csv(filename)
    lowered = [_.lower() for _ in uni.iloc[:, 0].tolist()]
    org2name = dict(zip(lowered, uni.iloc[:, 1].tolist()))
    org2country = dict(zip(lowered, uni.iloc[:, 2].tolist()))
    name2country = dict(zip(uni.iloc[:, 1], uni.iloc[:, 2].tolist()))
    return org2name, org2country, name2country

org2name, org2country, name2country = read_mapping()

def get_papers_for_name_and_country(affiliations, org2name, org2country):
    name2papers = ddict(set)
    country2papers = ddict(set)
    for org in affiliations:
        name = org2name.get(org.lower().strip(), org)
        country = org2country.get(org.lower().strip(), org)
        if name and name not in ['None', 'N/A']:
            name2papers[name] = name2papers[name].union(set(affiliations[org]))
            country2papers[country] = country2papers[country].union(set(affiliations[org]))
    return name2papers, country2papers

name2papers, country2papers = get_papers_for_name_and_country(affiliations, org2name, org2country)

def get_paper2name(paper2affiliations, org2name):
    paper2name = ddict(set)
    for paper, affs in paper2affiliations.items():
        paper2name[paper] = set([org2name.get(aff.lower().strip(), aff) for aff in affs])
    return paper2name

paper2name = get_paper2name(paper2affiliations, org2name)

def get_scores_per_name(paper2name):
    name2score = ddict(int)
    for paper, names in paper2name.items():
        for name in names:
            if name not in ['None', 'N/A']:
                name2score[name] += 1./len(names)
    return name2score

name2score = get_scores_per_name(paper2name)

def parse_proceedings(filename):
    names = ddict(list)
    
    with open(filename, encoding='utf') as f:
        for line in f:
            splitted = line.split(',')
            words = splitted[0].split()
            names[' '.join(words[-2:])].append(line)
            for chunk in splitted[1:]:
                names[chunk.strip()].append(line)
            
    return names 
            

In [ ]:
import pandas as pd

len_dict = {}
for key, val in country2papers.items():
    len_dict[key] = len(val)
#print(len(country2papers))

df = pd.DataFrame.from_dict(len_dict, orient = "index", columns = ["n_articles"])
df

,n_articles
USA,1186
UK,205
Intel,9
Adobe,9
France,64
...,...
Institute of Science,1
University of Texas - Austin,1
VIS,1
Swissquote,1


In [ ]:
pd.set_option('display.max_rows', None)


df.sort_values(by = "n_articles", axis=0, ascending=False) #.head(30)

,n_articles
USA,1186
China,259
UK,205
Canada,98
Switzerland,90
Germany,70
France,64
Israel,55
South Korea,53
Australia,53


In [ ]:
df_gdp = pd.read_csv('../country_gdp_population.csv')

In [ ]:
df['Name']=df.index.values
merged_df =  pd.merge(df, df_gdp, how="left", on="Name")

In [ ]:
merged_df = merged_df.dropna()

In [ ]:
merged_df["Articles_per_capita"] = merged_df["n_articles"] / merged_df["Population(Million)"]

In [ ]:
merged_df["Articles_over_GDP"] = merged_df["n_articles"] / merged_df["GDP"]

In [ ]:
merged_df["Articles_over_GDP_Per_Capita"] = merged_df["n_articles"] / (merged_df["GDP"]/merged_df["Population(Million)"])

In [ ]:
print(merged_df.to_string())

     n_articles            Name           GDP  Population(Million)  Articles_per_capita  Articles_over_GDP  Articles_over_GDP_Per_Capita
0          1186             USA  2.142770e+07               328.20             3.613650           0.000055                      0.018166
1           205              UK  2.827113e+06                66.65             3.075769           0.000073                      0.004833
4            64          France  2.715518e+06                67.06             0.954369           0.000024                      0.001580
5            98          Canada  1.736426e+06                37.59             2.607076           0.000056                      0.002121
8            70         Germany  3.845630e+06                83.02             0.843170           0.000018                      0.001511
12          259           China  1.434290e+07              1439.30             0.179949           0.000018                      0.025990
14           44       Singapore  3.720625

In [ ]:
merged_df.sort_values(by = "Articles_per_capita", axis=0, ascending=False).head(10)

,n_articles,Name,GDP,Population(Million),Articles_per_capita,Articles_over_GDP,Articles_over_GDP_Per_Capita
24,90,Switzerland,7.030824e+05,8.55,10.526316,0.000128,0.001094
14,44,Singapore,3.720625e+05,5.70,7.719298,0.000118,0.000674
27,55,Israel,3.950987e+05,9.05,6.077348,0.000139,0.001260
0,1186,USA,2.142770e+07,328.20,3.613650,0.000055,0.018166
1,205,UK,2.827113e+06,66.65,3.075769,0.000073,0.004833
102,15,Finland,2.687612e+05,5.52,2.717391,0.000056,0.000308
5,98,Canada,1.736426e+06,37.59,2.607076,0.000056,0.002121
16,53,Australia,1.392681e+06,25.36,2.089905,0.000038,0.000965
33,11,Denmark,3.480780e+05,5.81,1.893287,0.000032,0.000184
84,14,Sweden,5.308329e+05,10.23,1.368524,0.000026,0.000270


In [ ]:
merged_df.sort_values(by = "Articles_over_GDP", axis=0, ascending=False).head(10)

,n_articles,Name,GDP,Population(Million),Articles_per_capita,Articles_over_GDP,Articles_over_GDP_Per_Capita
27,55,Israel,3.950987e+05,9.05,6.077348,0.000139,0.001260
24,90,Switzerland,7.030824e+05,8.55,10.526316,0.000128,0.001094
14,44,Singapore,3.720625e+05,5.70,7.719298,0.000118,0.000674
1,205,UK,2.827113e+06,66.65,3.075769,0.000073,0.004833
5,98,Canada,1.736426e+06,37.59,2.607076,0.000056,0.002121
102,15,Finland,2.687612e+05,5.52,2.717391,0.000056,0.000308
0,1186,USA,2.142770e+07,328.20,3.613650,0.000055,0.018166
322,1,Cyprus,2.456465e+04,0.88,1.136364,0.000041,0.000036
16,53,Australia,1.392681e+06,25.36,2.089905,0.000038,0.000965
28,53,South Korea,1.642383e+06,51.71,1.024947,0.000032,0.001669


In [ ]:
merged_df.sort_values(by = "Articles_over_GDP_Per_Capita", axis=0, ascending=False).head(10)

,n_articles,Name,GDP,Population(Million),Articles_per_capita,Articles_over_GDP,Articles_over_GDP_Per_Capita
12,259,China,1.434290e+07,1439.30,0.179949,0.000018,0.025990
0,1186,USA,2.142770e+07,328.20,3.613650,0.000055,0.018166
34,24,India,2.875142e+06,1380.00,0.017391,0.000008,0.011519
1,205,UK,2.827113e+06,66.65,3.075769,0.000073,0.004833
5,98,Canada,1.736426e+06,37.59,2.607076,0.000056,0.002121
28,53,South Korea,1.642383e+06,51.71,1.024947,0.000032,0.001669
4,64,France,2.715518e+06,67.06,0.954369,0.000024,0.001580
8,70,Germany,3.845630e+06,83.02,0.843170,0.000018,0.001511
27,55,Israel,3.950987e+05,9.05,6.077348,0.000139,0.001260
360,3,Vietnam,2.619212e+05,96.46,0.031101,0.000011,0.001105
